
# Notebook 03 - Feature Engineering & Multimodal Inputs

**Project:** Recognizing the Unseen - A Multimodal, Trauma-Informed AI Framework 
**Goal of this notebook:** engineer features beyond PHQ-8 and prepare multimodal inputs (text, audio, video) for downstream modeling.

**Builds on:** 
- Notebook 01: Import, clean, EDA (labels + minimal cleaning) 
- Notebook 02: Baselines (Dummy vs. Logistic), ROC/PR, coefficient plots, interactive sliders + thresholds 


---
## Contents
1. Data sources & setup 
2. SMT guardrails (Z3) for data integrity and split hygiene 
3. Feature engineering 
 - Tabular (PHQ-8) 
 - Text (transcripts embeddings) 
 - Audio (prosody) 
 - Video (facial action units) 
4. Multimodal dataset assembly 
5. Artifacts (saved processed data) 
6. Limitations & reproducibility 
7. Closing summary & next steps
---


## 1) Data sources & setup

Load the cleaned PHQ-8 labels and set up placeholders for additional modalities. 
This cell focuses on reading already-prepared artifacts from prior notebooks and defining
conventions for participant/session keys.


In [ ]:
# =============================================================================
# Imports & Paths  +  Data sources & setup (canonical one-stop block)
# -----------------------------------------------------------------------------
# - Imports (numpy/pandas, etc.) + quick diagnostics (Python/pandas version)
# - Resolve repo root and standard data directories
# - Define canonical join/target names used across Notebook 03
# - Load cleaned labels (from Notebook 01/02 output) with graceful fallback
# - Normalize common column-name variants -> {participant_id, label, split}
# - Print a concise summary for reviewers
# =============================================================================

# --- Imports -----------------------------------------------------------------
from pathlib import Path
import pandas as pd
import numpy as np

# (Optional) quick diagnostics to make runtime context explicit
import platform
print("Python:", platform.python_version())
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
# Optional: show whether any BLAS info is available (won't crash if missing)
blas = getattr(getattr(np, "__config__", object()), "blas_opt_info", {})
print("BLAS info found:", bool(blas))

# --- Paths (single source of truth) ------------------------------------------
# Reuse ROOT_DIR if already set (e.g., by earlier cells); otherwise derive it.
try:
    ROOT_DIR
except NameError:
    ROOT_DIR = Path.cwd().resolve().parent  # notebooks/ -> repo root

DATA_DIR      = ROOT_DIR / "data"
CLEAN_DIR     = DATA_DIR / "cleaned"       # outputs from NB01/NB02 (adjust if your layout differs)
PROCESSED_DIR = DATA_DIR / "processed"     # downstream features go here
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# --- Canonical names & expected labels artifact ------------------------------
JOIN_KEY = "participant_id"                # canonical join key used throughout NB03
TARGET   = "label"                         # canonical binary target (0/1)
LABELS_PATH = CLEAN_DIR / "labels_clean.parquet"   # <- adjust here if NB01/02 writes elsewhere

# --- Load labels with guardrails (degrade gracefully if file not ready) ------
if LABELS_PATH.exists():
    labels_df = pd.read_parquet(LABELS_PATH)
    print(f"Loaded labels: {LABELS_PATH} | shape={labels_df.shape}")
else:
    print(f'NOTE: labels_clean.parquet not found -> {LABELS_PATH}')
    print("      (Run Notebook 01/02 to generate, or adjust LABELS_PATH if it moved.)")
    # Create empty skeleton so downstream checks in Step 2 can SKIP cleanly
    labels_df = pd.DataFrame(columns=[JOIN_KEY, TARGET, "split"])

# --- Normalize column names to our canonical schema --------------------------
rename_map = {}
# join key variants
if "subject_id" in labels_df.columns and JOIN_KEY not in labels_df.columns:
    rename_map["subject_id"] = JOIN_KEY
if "id" in labels_df.columns and JOIN_KEY not in labels_df.columns:
    rename_map["id"] = JOIN_KEY
# target variants
if "target" in labels_df.columns and TARGET not in labels_df.columns:
    rename_map["target"] = TARGET
if "phq8_binary" in labels_df.columns and TARGET not in labels_df.columns:
    rename_map["phq8_binary"] = TARGET

if rename_map:
    labels_df = labels_df.rename(columns=rename_map)

# Ensure placeholders exist (keeps downstream guardrails readable & safe)
if JOIN_KEY not in labels_df.columns:
    labels_df[JOIN_KEY] = pd.Series(dtype="object")
    print(f'NOTE: added empty "{JOIN_KEY}" column')
if TARGET not in labels_df.columns:
    labels_df[TARGET] = pd.Series(dtype="Int64")  # nullable int; matches {0,1}
    print(f'NOTE: added empty "{TARGET}" column')
if "split" not in labels_df.columns:
    labels_df["split"] = pd.Series(dtype="string")
    print('NOTE: added empty "split" column')

# --- Reviewer-friendly summary ----------------------------------------------
n_rows = len(labels_df)
sample_cols = [c for c in [JOIN_KEY, TARGET, "split"] if c in labels_df.columns]
print(f"labels_df: {n_rows} rows, {labels_df.shape[1]} cols | has {sample_cols} | head:")
print(labels_df[sample_cols].head(5))


---
## 2) SMT guardrails (Z3) for data integrity and split hygiene

We add lightweight **formal checks** to catch structural mistakes early:

- Temporal event sanity: `onset < apex < offset n_frames - 1` 
- Window safety: each feature window stays within clip bounds 
- Sampling consistency: `fps > 0` and `duration frames / fps` 
- Split hygiene: subject-disjoint train/val/test; minimum class presence per split 
- Label domain checks: labels belong to the expected set
---

In [ ]:
# =============================================================================
# STEP 2 - SMT GUARDRAILS (Z3) + SPLIT HYGIENE
# Goal:
#   - Enforce label domain and (optionally) split integrity with small, readable
#     checks that fail-fast when assumptions break.
#   - Keep notebook executable even when artifacts are not ready (print & skip).
# Why:
#   - Early structural checks catch silent drift (e.g., wrong label domain, ID
#     overlap across splits) before modeling.
# =============================================================================

# Make repo-root imports work from inside notebooks/
# Why: the kernel's CWD is often `notebooks/`, while `verification.py` lives at repo root.
import sys
from pathlib import Path

def _find_repo_root(filename: str = "verification.py") -> Path | None:
    """Walk upward from CWD until `filename` is found; return its parent (repo root)."""
    here = Path.cwd().resolve()
    for p in [here, *here.parents]:
        if (p / filename).exists():
            return p
    return None

# Reuse ROOT_DIR from Step 1 if present; otherwise resolve it robustly here.
try:
    ROOT_DIR
except NameError:
    ROOT_DIR = _find_repo_root() or Path.cwd().resolve().parent  # fallback: notebooks/ -> repo root

# Ensure the root is importable
if ROOT_DIR and str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

# Now safe(ish) to import guardrail utilities; if unavailable, degrade gracefully.
_guardrails_loaded = False
try:
    from verification import (
        check_event_triplet,             # example timing check for onset/apex/offset
        check_window_bounds,             # window [start, start+len) within [0, n)
        check_sampling_consistency,      # duration  frames/fps
        assert_disjoint_splits,          # no subject overlap across splits
        min_class_presence,              # per-split class counts  threshold
        assert_label_domain,             # labels in allowed set
        verify_env,                      # tiny runtime report for smoke tests/CI
    )
    _guardrails_loaded = True
except Exception as e:
    print(f"SKIP: guardrail utilities not importable ({type(e).__name__}: {e}). "
          "Proceeding without hard checks so the notebook stays runnable.")

JOIN_KEY = "participant_id"
TARGET   = "label"

# ---- 2.1 Label hygiene ------------------------------------------------------
if not _guardrails_loaded:
    print("SKIP: label checks (verification.py not loaded).")
elif "labels_df" not in globals() or labels_df is None or labels_df.empty or TARGET not in labels_df.columns:
    print("SKIP: label checks (labels_df empty or target column missing).")
else:
    # Domain guarantee  reviewers see intent: binary classification (0/1).
    assert_label_domain(labels_df[TARGET], allowed=(0, 1))
    print("OK: label domain is restricted to {0, 1}.")

# ---- 2.2 Split hygiene (optional) -------------------------------------------
# If you already created a split in Notebook 02, this validates it.
if not _guardrails_loaded:
    print("SKIP: split checks (verification.py not loaded). "
          "Create deterministic splits in Notebook 02/03 before modeling.")
elif ("labels_df" in globals() and labels_df is not None and not labels_df.empty
      and ("split" in labels_df.columns) and (JOIN_KEY in labels_df.columns)):
    # Extract subject IDs per split (keeps checks explainable & auditable).
    train_ids = labels_df.loc[labels_df["split"] == "train", JOIN_KEY]
    val_ids   = labels_df.loc[labels_df["split"] == "val",   JOIN_KEY]
    test_ids  = labels_df.loc[labels_df["split"] == "test",  JOIN_KEY]

    # (a) No subject overlap across splits
    assert_disjoint_splits(train_ids, val_ids, test_ids)
    print("OK: no subject overlap across splits (train/val/test).")

    # (b) Minimum per-class support in each split  guards against degenerate folds
    min_class_presence(
        {
            "train": labels_df.loc[labels_df["split"] == "train", TARGET],
            "val":   labels_df.loc[labels_df["split"] == "val",   TARGET],
            "test":  labels_df.loc[labels_df["split"] == "test",  TARGET],
        },
        min_count=5  # Adjust with dataset size; aim to preserve evaluation stability.
    )
    print("OK: each split meets minimum class presence thresholds.")
else:
    print('SKIP: split checks (no "split" column yet). '
          "Create deterministic splits in Notebook 02/03 before modeling.")

# ---- 2.3 Timing/window sanity (optional, runs only if variables provided) ---
# These are examples; they will quietly skip if you haven't defined the inputs yet.
# Rationale: keeps nbconvert/CI green while still documenting expectations.

if _guardrails_loaded:
    # Example A: sampling consistency for a video segment: frames / fps  duration
    try:
        ok, msg = check_sampling_consistency(
            frames=int(video_frames),        # define upstream when available
            fps=float(video_fps),
            duration_sec=float(video_duration_sec)
        )
        print("Video sampling check:", msg)
    except Exception:
        # Not available yet; that is expected in early drafts.
        pass

    # Example B: generic window bounds (e.g., feature extraction slices)
    try:
        ok, msg = check_window_bounds(
            start=int(win_start),            # define upstream when available
            length=int(win_len),
            n_frames=int(total_frames)
        )
        print("Window bounds check:", msg)
    except Exception:
        pass
else:
    print("SKIP: timing/window checks (verification.py not loaded).")

print("Guardrail checks completed.")

# =============================================================================
# Smoke test - confirm guardrail utilities are importable and show env facts
# =============================================================================
_loaded = globals().get("_guardrails_loaded", False)

if _loaded:
    try:
        import verification
        print(f"Verification module loaded from: {verification.__file__}")
        want = [
            "check_event_triplet",
            "check_window_bounds",
            "check_sampling_consistency",
            "assert_disjoint_splits",
            "min_class_presence",
            "assert_label_domain",
            "verify_env",
        ]
        available = [name for name in want if getattr(verification, name, None)]
        missing   = [name for name in want if name not in available]
        print("Available guardrail functions:", available)
        if missing:
            print("Note: missing in verification.py ->", missing)
        # One-line environment report (nice for CI and Dr. S)
        try:
            print("Env:", verification.verify_env())
        except Exception:
            print("Env: verify_env() raised; skipping.")
    except Exception as e:
        print(f"Smoke test warning: import succeeded but inspection failed ({type(e).__name__}: {e})")
else:
    print("Smoke test: verification.py not loaded (see SKIP messages above).")

# Show where ROOT_DIR resolved to (useful for CI/review logs)
print("Resolved ROOT_DIR:", ROOT_DIR if "ROOT_DIR" in globals() else "<not set>")

# Peek at the first few sys.path entries to confirm import order
print("sys.path[0:3]:", sys.path[:3])






> 💡 **Workflow tip:** Run the checks immediately after loading each modality. Fail fast with clear errors so issues don't propagate into modeling.


---
## 3) Feature engineering
We create modality-specific features. Start simple and keep everything **reproducible**.

### 3.1 Tabular (PHQ-8)
- Standardize numeric PHQ-8 items.
- (Optional) Create low-order interaction terms for hypothesis-driven pairs.
---

In [ ]:
# Quick probe: see all columns that look PHQ-related
[c for c in labels_df.columns if "phq" in str(c).lower()]


In [ ]:
# =============================================================================
# 3.1 Tabular (PHQ-8) - Clinical-style imputation + optional rounding
# Goal:
#   - Build interpretable PHQ-8 features (sum/mean/missingness, z-scores).
#   - Clinical scoring:
#       * If 1 item missing  impute that item with the row mean, then sum.
#       * If 2 items missing  leave score NaN (no aggressive imputation).
#   - Optional rounding of the final score to match reporting conventions.
#   - After scoring, zero-fill item columns for downstream models (documented).
# =============================================================================

import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

TAB_OUT = PROCESSED_DIR / "tabular_phq8.parquet"

# ---- Explicit PHQ-8 schema pin (order matters: items 1..8) ------------------
PHQ8_COLS = [
    "phq8_nointerest",      # 1  little interest/pleasure
    "phq8_depressed",       # 2  feeling down/depressed/hopeless
    "phq8_sleep",           # 3  sleep problems
    "phq8_tired",           # 4  low energy/tired
    "phq8_appetite",        # 5  appetite/eating
    "phq8_failure",         # 6  feeling bad/failure/worthless/guilty
    "phq8_concentrating",   # 7  trouble concentrating
    "phq8_moving",          # 8  psychomotor (restless/slow)
]
REQUIRE_ALL_ITEMS = True  # set False to proceed if any items missing

# Choose rounding for the total score: "nearest" | "bankers" | "floor" | "ceil" | None
SCORE_ROUNDING = "nearest"

# ---- Guard schema presence ---------------------------------------------------
missing_items = [c for c in PHQ8_COLS if c not in labels_df.columns]
if missing_items:
    msg = f"PHQ-8 schema mismatch: missing {len(missing_items)} column(s): {missing_items}"
    if REQUIRE_ALL_ITEMS:
        raise AssertionError(msg)
    else:
        print("WARNING:", msg, " proceeding with available items only.")
        PHQ8_COLS = [c for c in PHQ8_COLS if c in labels_df.columns]

if not PHQ8_COLS:
    print("SKIP: No PHQ-8 item columns available; tabular features will be empty.")
    tab_df = pd.DataFrame(columns=[JOIN_KEY, TARGET])
else:
    # ---- 3.1.1 Assemble base frame ------------------------------------------
    base_cols = [c for c in [JOIN_KEY, TARGET] if c in labels_df.columns]
    tab_df = labels_df[base_cols + PHQ8_COLS].copy()

    # Coerce items to numeric safely (handles stray strings gracefully)
    items = tab_df[PHQ8_COLS].apply(pd.to_numeric, errors="coerce")

    # ---- 3.1.2 Clinical-style imputation & scoring ---------------------------
    missing_ct = items.isna().sum(axis=1)        # items missing per row
    row_mean   = items.mean(axis=1, skipna=True) # mean of answered items

    # Impute only when exactly 1 (or 1) item missing
    items_imputed = items.copy()
    mask_impute = missing_ct.le(1) & missing_ct.gt(0)  # (0 < missing  1)
    items_imputed.loc[mask_impute] = (
        items_imputed.loc[mask_impute].T
        .fillna(row_mean[mask_impute])  # broadcast row-wise means into NaNs
        .T
    )

    # Score:
    #  - If 2 items missing  keep NaN (min_count enforces that)
    #  - Else  sum imputed row
    tab_df["phq8_missing_count"] = missing_ct
    tab_df["phq8_sum"]  = items_imputed.sum(axis=1, min_count=len(PHQ8_COLS) - 1)
    tab_df["phq8_mean"] = items_imputed.mean(axis=1, skipna=True)

    # ---- 3.1.3 Optional rounding to match reporting conventions -------------
    if SCORE_ROUNDING == "nearest":
        s = tab_df["phq8_sum"]
        tab_df["phq8_sum"] = np.sign(s) * np.floor(np.abs(s) + 0.5)  # half-away-from-zero
    elif SCORE_ROUNDING == "bankers":
        tab_df["phq8_sum"] = tab_df["phq8_sum"].round(0)
    elif SCORE_ROUNDING == "floor":
        tab_df["phq8_sum"] = np.floor(tab_df["phq8_sum"])
    elif SCORE_ROUNDING == "ceil":
        tab_df["phq8_sum"] = np.ceil(tab_df["phq8_sum"])
    # else: leave fractional totals as-is

    # ---- 3.1.4 Post-scoring zero-fill for model inputs (documented choice) ---
    # Keeps rows dense for models while preserving clinically faithful 'phq8_sum'.
    tab_df[PHQ8_COLS] = items.fillna(0)

    # ---- 3.1.5 Standardize numeric features (excluding target & ID) ----------
    num_cols = [c for c in tab_df.columns
                if c not in [JOIN_KEY, TARGET] and pd.api.types.is_numeric_dtype(tab_df[c])]
    if num_cols:
        scaler = StandardScaler()
        tab_df[[f"{c}_z" for c in num_cols]] = scaler.fit_transform(tab_df[num_cols])
        print(f"Scaled {len(num_cols)} numeric columns -> *_z")
    else:
        print("NOTE: No numeric columns to scale.")

# ---- 3.1.6 Save & reviewer preview -----------------------------------------
try:
    tab_df.to_parquet(TAB_OUT, index=False)
    print("Saved tabular PHQ-8 ->", TAB_OUT, "| shape=", tab_df.shape)
except Exception as e:
    print("SKIP save:", type(e).__name__, "-", e)

show_cols = [JOIN_KEY, TARGET] + PHQ8_COLS + ["phq8_missing_count", "phq8_sum", "phq8_mean"]
show_cols = [c for c in show_cols if c in tab_df.columns]
print("tab_df preview:")
print(tab_df[show_cols].head(5))

# ---- Optional QA against any provided 'phq8_score' column -------------------
if "phq8_score" in labels_df.columns:
    try:
        orig = pd.to_numeric(labels_df["phq8_score"], errors="coerce")
        agree = (orig.fillna(-1).astype(float) == tab_df["phq8_sum"].fillna(-2).astype(float)).sum()
        print(f"QA: phq8_sum (clinical + rounding) vs phq8_score agreement: {agree}/{len(tab_df)} rows")
    except Exception:
        print("QA: could not compare to 'phq8_score' (non-fatal).")




   

---
## 🕷️ Spider Check - PHQ-8 Tabular Peek

Anchoring in ground truth: survey responses and labels.  
A simple check that PHQ-8 scores and symptom counts align with expectations.  

***Because anchors keep us steady when we search for the unseen.***

---


In [ ]:
# =============================================================================
# PHQ-8 QA (optional): compare our phq8_sum to provided phq8_score
# =============================================================================
if "phq8_score" in labels_df.columns and "phq8_sum" in tab_df.columns:
    orig = pd.to_numeric(labels_df["phq8_score"], errors="coerce")
    ours = pd.to_numeric(tab_df["phq8_sum"], errors="coerce")

    mismask = orig.fillna(-1).astype(float) != ours.fillna(-2).astype(float)
    mism_idx = mismask[mismask].index
    n_mis = int(mismask.sum())

    print(f"QA: mismatches (ours vs provided): {n_mis}/{len(tab_df)} rows")
    if n_mis:
        cols = [JOIN_KEY, "phq8_score", "phq8_sum", "phq8_mean", "phq8_missing_count"] + PHQ8_COLS
        # Show up to 5 examples
        preview = tab_df.loc[mism_idx, [c for c in cols if c in tab_df.columns]].head(5).copy()
        # Add the provided score for clarity (from labels_df)
        preview["phq8_score_src"] = labels_df.loc[preview.index, "phq8_score"]
        display(preview)
else:
    print("QA: skipped (no 'phq8_score' column or 'phq8_sum' not computed).")




---
### PHQ-8 tabular features: interpretation & key takeaways

**What we did**
- Pinned PHQ-8 item schema: ["phq8_nointerest","phq8_depressed","phq8_sleep","phq8_tired","phq8_appetite","phq8_failure","phq8_concentrating","phq8_moving"].
- Clinical-style scoring:
  - If 1 item missing: imputed the missing item with the row mean of answered items, then summed.
  - If 2 items missing: left the score as NaN (no aggressive imputation).
- Optional rounding: set to "nearest" so totals match typical reporting.
- After scoring, zero-filled item columns for modeling, and z-scored numeric features for comparability.

**Guardrails & QA**
- Label domain and split checks run in Step 2 (fail-fast or SKIP cleanly).
- PHQ-8 QA: our computed "phq8_sum" vs provided "phq8_score"  **107/107** agreement with rounding ("nearest").

**Results snapshot**
- Saved to `data/processed/tabular_phq8.parquet`.
- Shape: **(107, 24)** (ID, label, 8 items, missing_count, sum, mean, and z-scored variants).
- Missingness: `phq8_missing_count` shows per-row item gaps; rows with 2 missing keep `phq8_sum` as NaN.

**How to read the features**
- `phq8_sum`: total symptom burden (higher = more severe).
- `phq8_mean`: average per-item severity (robust when one item is imputed).
- `phq8_missing_count`: data quality indicator; consider as a covariate or filter in sensitivity analyses.
- `*_z`: standardized versions for models that benefit from scaled inputs.

**Decisions (documented)**
- Rounding: used "nearest" to mirror the provided clinical scores (prevents off-by-one drift when one item is imputed).
- Post-scoring zero-fill: keeps downstream models dense without altering the clinically faithful `phq8_sum`.

**Limitations**
- Row-mean imputation for a single missing item is simple and standard, but still an assumption.
- Rows with 2 missing items are not scored; downstream models should either ignore `phq8_sum` for those rows or handle NaNs explicitly.

**Recommended next steps**
- Sensitivity check: run models with and without rounding; confirm conclusions are stable.
- Optionally add `phq8_flag_gt1_missing = 1{missing_count  2}` as an exclusion flag or covariate.
- Proceed to 3.2 (Text) to add linguistic signals; the tabular block provides a solid baseline.
---



### 3.2 Text (transcripts embeddings)
- Option A (quick baseline): TF IDF on transcript text. 
- Option B (semantic): sentence embeddings (e.g., SentenceTransformers).

> Note: If running offline or with limited resources, prefer TF IDF first; swap in embeddings later.
> 
---

In [ ]:
# =============================================================================
# 3.2a Import transcripts (DAIC/AVEC-style) and join into labels_df  - ROBUST
# =============================================================================
from pathlib import Path
import pandas as pd
import re

RAW_DIR = ROOT_DIR / "data"

# Reuse join key safely (in case cells ran out of order)
JOIN_KEY = globals().get("JOIN_KEY", "participant_id")

tx_files = list(RAW_DIR.rglob("*_TRANSCRIPT.csv"))
print(f"Found {len(tx_files)} transcript file(s) under {RAW_DIR}")

def _participant_id_from_stem(stem: str) -> str:
    m = re.match(r"^(\d+)", stem)
    return m.group(1) if m else stem

def _read_transcript_csv(path: Path) -> pd.DataFrame | None:
    """Try several parsers to handle comma/tab and odd encodings."""
    for kwargs in (
        {"engine": "python", "sep": None},     # sniff delimiter
        {"sep": "\t"},                         # tab-separated
        {"sep": ","},                          # comma-separated
        {"engine": "python", "sep": r"\s+"},   # any whitespace
    ):
        for enc in ("utf-8", "latin-1"):
            try:
                return pd.read_csv(path, encoding=enc, **kwargs)
            except Exception:
                continue
    print(f"SKIP: could not read {path.name} with common parsers")
    return None

# preferred text/speaker header names (case-insensitive)
TEXT_CANDIDATES    = ["transcript", "value", "text", "utterance", "content"]
SPEAKER_CANDIDATES = ["speaker", "speaker_id"]

rows = []
for p in tx_files:
    df = _read_transcript_csv(p)
    if df is None or df.empty:
        print(f"SKIP: empty or unreadable -> {p.name}")
        continue

    # If pandas mis-parsed delimiter, you might see a single big column with brackets.
    # Split that if needed (rare, but seen in weird exports).
    if len(df.columns) == 1 and df.columns[0].strip().startswith("[") and "," in df.columns[0]:
        # Try to split header string into real columns
        raw = df.columns[0]
        cols = [c.strip(" '\"") for c in raw.strip("[]").split(",")]
        df = df.rename(columns={df.columns[0]: cols[0]})
        # No rows to split; most of the time this case doesn't have usable data.
        print(f"SKIP: header looked bundled in {p.name} -> columns recovered: {cols}")

    lower_map = {str(c).lower(): c for c in df.columns}
    text_col = next((lower_map[c] for c in TEXT_CANDIDATES if c in lower_map), None)
    if text_col is None:
        print(f"SKIP: no recognizable text column in {p.name} (cols={list(df.columns)[:10]})")
        continue

    speaker_col = next((lower_map[c] for c in SPEAKER_CANDIDATES if c in lower_map), None)
    if speaker_col is not None:
        keep = df[speaker_col].astype(str).str.lower().isin(
            ["participant", "p", "subject", "interviewee", "patient"]
        )
        if keep.any():
            df = df.loc[keep]

    text = (
        df[text_col]
        .astype(str).fillna("")
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
        .tolist()
    )
    transcript = " ".join([t for t in text if t])

    part_id = _participant_id_from_stem(p.stem)
    rows.append({"participant_id": part_id, "transcript": transcript})

tx_df = pd.DataFrame(rows)
print("Built transcripts table:", tx_df.shape)

# Save unified for provenance
TRANSCRIPTS_UNIFIED = PROCESSED_DIR / "transcripts_unified.csv"
tx_df.to_csv(TRANSCRIPTS_UNIFIED, index=False)
print("Wrote unified transcripts ->", TRANSCRIPTS_UNIFIED)

# Merge into labels_df on JOIN_KEY, with safe dtype casting only if the columns exist
if JOIN_KEY in labels_df.columns and not tx_df.empty:
    labels_df[JOIN_KEY] = labels_df[JOIN_KEY].astype(str)
    tx_df["participant_id"] = tx_df["participant_id"].astype(str)
    before_cols = labels_df.shape[1]
    labels_df = labels_df.merge(
        tx_df.rename(columns={"participant_id": JOIN_KEY}), on=JOIN_KEY, how="left"
    )
    print(f"Merged transcripts into labels_df: columns {before_cols} -> {labels_df.shape[1]}")
    print("labels_df now has 'transcript':", "transcript" in labels_df.columns)
elif JOIN_KEY not in labels_df.columns:
    print(f"SKIP merge: JOIN_KEY '{JOIN_KEY}' not present in labels_df.columns={list(labels_df.columns)[:10]}")
else:
    print("No transcripts constructed; 3.2 will SKIP safely.")





---
## 🕷️ Spider Check - Text Meta Peek

Word footprints: character counts, tokens, sentence lengths.  
A tiny check that transcripts really hold the shape we expect before diving deeper.  

***Because even footprints tell a story of the unseen.*** 

---


In [ ]:
# Inspect any transcript-like columns
tx_like = [c for c in labels_df.columns if "transcript" in c.lower()]
print("Transcript-like columns:", tx_like)

# Coalesce to a single 'transcript' column (handles _x/_y cases)
if "transcript" not in labels_df.columns:
    cand_x = next((c for c in tx_like if c.endswith("_x")), None)
    cand_y = next((c for c in tx_like if c.endswith("_y")), None)
    cand_plain = next((c for c in tx_like if c == "transcript"), None)

    src = cand_plain or cand_x or cand_y
    if src:
        labels_df["transcript"] = labels_df[src]
        # drop the extra copies if present
        for c in set(tx_like) - {"transcript"}:
            labels_df.drop(columns=c, inplace=True, errors="ignore")

print("Has 'transcript' now:", "transcript" in labels_df.columns)
print("Non-null transcripts:", int(labels_df["transcript"].notna().sum()) if "transcript" in labels_df.columns else 0)


In [ ]:
# =============================================================================
# 3.2 Text (transcripts) - TF-IDF baseline + lightweight QC (SKIP-safe)
# -----------------------------------------------------------------------------
# Goal:
#    Provide a fast, interpretable textual signal using TF-IDF on transcripts.
#    Add simple QC features (length in chars/tokens, sentence count) to inspect data quality.
#    Degrade gracefully when transcripts are not available (print + write placeholders).
#    Save artifacts to processed/ for later multimodal joins.
# Why:
#    TF-IDF gives a transparent baseline before heavier embeddings.
#    QC features help reviewers see whether text length/coverage varies by subject/split.
# =============================================================================

import re, json
import numpy as np
import pandas as pd
from pathlib import Path

# ---- 3.2.0 Output locations (consistent with 3.1/Section 5) -----------------
TEXT_TFIDF_OUT = PROCESSED_DIR / "text_tfidf.parquet"   # JOIN_KEY + tfidf_* columns (dense float32)
TEXT_META_OUT  = PROCESSED_DIR / "text_meta.parquet"    # JOIN_KEY + QC features
TEXT_VOCAB_OUT = PROCESSED_DIR / "text_tfidf_vocab.json"

# ---- 3.2.1 Vectorizer settings (balanced for speed + signal) ----------------
TFIDF_MAX_FEATS = 2048        # cap features for speed & dimensionality control
TFIDF_NGRAMS    = (1, 2)      # unigrams + bigrams capture short cues/phrases
TFIDF_MIN_DF    = 2           # drop terms that appear in only one document

# ---- 3.2.2 Find the transcript column in labels_df --------------------------
# We prefer an explicit 'transcript' column, but accept common variants or
# any column containing the word 'transcript' (case-insensitive).
TRANSCRIPT_CANDIDATES = ["transcript", "transcript_text", "text", "utterance", "asr_text"]

def _find_transcript_column(df: pd.DataFrame) -> str | None:
    lower = {str(c).lower(): c for c in df.columns}
    # exact matches first (more predictable)
    for name in TRANSCRIPT_CANDIDATES:
        if name in lower:
            return lower[name]
    # fallback: any column whose name contains 'transcript'
    for k, orig in lower.items():
        if "transcript" in k:
            return orig
    return None

tx_col = _find_transcript_column(labels_df)

if tx_col is None:
    # No transcripts yet  write empty placeholders so later joins don't break
    print("SKIP: No transcript column found. Looked for:", TRANSCRIPT_CANDIDATES)
    pd.DataFrame(columns=[JOIN_KEY]).to_parquet(TEXT_TFIDF_OUT, index=False)
    pd.DataFrame(columns=[JOIN_KEY]).to_parquet(TEXT_META_OUT, index=False)
    Path(TEXT_VOCAB_OUT).write_text(json.dumps({"vocab": [], "ngram_range": TFIDF_NGRAMS, "min_df": TFIDF_MIN_DF}))
else:
    # ---- 3.2.3 Assemble/normalize text frame --------------------------------
    # Keep only ID, target, and the transcript column (keeps artifacts small & deterministic)
    base_cols = [c for c in [JOIN_KEY, TARGET, tx_col] if c in labels_df.columns]
    text_df = labels_df[base_cols].copy()

    # Normalize text lightly (stable across platforms; preserves punctuation for tokens)
    text_df[tx_col] = (
        text_df[tx_col].astype("string").fillna("")
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

    # ---- 3.2.4 QC features (length in chars/tokens, sentence count) ----------
    # Why: quick sanity to spot empty/short transcripts or outliers by subject/split.
    def _tokenize(s: str) -> list[str]:
        return re.findall(r"\b[\w'-]+\b", s.lower())

    meta = text_df[[JOIN_KEY]].copy()
    meta["text_len_chars"]      = text_df[tx_col].str.len().astype("Int64")
    meta["text_len_tokens"]     = text_df[tx_col].apply(lambda s: len(_tokenize(s))).astype("Int64")
    meta["text_num_sentences"]  = text_df[tx_col].str.count(r"[.!?]").astype("Int64")
    meta.to_parquet(TEXT_META_OUT, index=False)
    print(f"Saved text meta -> {TEXT_META_OUT} | shape={meta.shape}")

    # ---- 3.2.5 TF-IDF construction (only for nonempty transcripts) ----------
    nonempty = text_df[text_df[tx_col].str.len() > 0]
    if nonempty.empty:
        print(f"SKIP: transcript column '{tx_col}' is present but all rows are empty; TF-IDF not built.")
        pd.DataFrame(columns=[JOIN_KEY]).to_parquet(TEXT_TFIDF_OUT, index=False)
        Path(TEXT_VOCAB_OUT).write_text(json.dumps({"vocab": [], "ngram_range": TFIDF_NGRAMS, "min_df": TFIDF_MIN_DF}))
    else:
        try:
            from sklearn.feature_extraction.text import TfidfVectorizer
        except Exception as e:
            # Keep notebook runnable even if sklearn isn't installed
            print("SKIP: scikit-learn not available for TF-IDF:", type(e).__name__, "-", e)
            pd.DataFrame(columns=[JOIN_KEY]).to_parquet(TEXT_TFIDF_OUT, index=False)
            Path(TEXT_VOCAB_OUT).write_text(json.dumps({"vocab": [], "ngram_range": TFIDF_NGRAMS, "min_df": TFIDF_MIN_DF}))
        else:
            vec = TfidfVectorizer(
                max_features=TFIDF_MAX_FEATS,
                ngram_range=TFIDF_NGRAMS,
                min_df=TFIDF_MIN_DF,
                stop_words="english",
                strip_accents="unicode",
                dtype=np.float32,

            )
            X = vec.fit_transform(nonempty[tx_col].tolist())
            vocab = vec.get_feature_names_out().tolist()
            tfidf_cols = [f"tfidf_{t}" for t in vocab]

            # Parquet does not support pandas Sparse by default  densify to float32
            arr = X.toarray().astype("float32")
            tfidf_df = pd.DataFrame(arr, columns=tfidf_cols)
            tfidf_df.insert(0, JOIN_KEY, nonempty[JOIN_KEY].to_numpy())

            # Save compressed parquet + sidecar vocab (for reproducibility)
            tfidf_df.to_parquet(TEXT_TFIDF_OUT, index=False, engine="pyarrow", compression="snappy")
            Path(TEXT_VOCAB_OUT).write_text(json.dumps(
                {"vocab": vocab, "ngram_range": TFIDF_NGRAMS, "min_df": TFIDF_MIN_DF}
            ))
            print(f"Saved TF-IDF -> {TEXT_TFIDF_OUT} | shape={tfidf_df.shape}")
            print(f"Saved TF-IDF vocab -> {TEXT_VOCAB_OUT} | {len(vocab)} terms")





In [ ]:
[c for c in labels_df.columns if any(k in str(c).lower() for k in ["transcript","text","utter","asr","notes","summary","content"])]


---
## 🕷️ Spider Check - TF-IDF Sanity Peek

Before building further, I pause for a **Spider Check**:  
like pulling back the covers in a cabin to make sure there are no critters,  
I peek into my data with a quick `head(2)` or shape check.  

It's not just a sanity step - it's a peace-of-mind ritual.  
***Because good data science isn't just about seeing what's obvious -  
it's about seeing the unseen.***

---




In [ ]:
# =============================================================================
# 3.2b Spider check - TF-IDF IDF transparency peek (read-only)
# -----------------------------------------------------------------------------
# What this does:
#    Reports how many participants have non-empty transcripts
#    Re-fits a TF-IDF vectorizer (same settings) purely to read IDF weights
#    Prints top/bottom IDF terms (rarest/most common)
#    Flags "spidery" terms (very short or odd chars) as a quick noise check
#    Saves a CSV of all terms+IDF for provenance: data/processed/text_idf_terms.csv
# Why separate from 3.2:
#    Keeps feature-building cell clean; this is an inspection-only "peek"
# =============================================================================

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from pathlib import Path

# 0) Guard: do we even have transcripts merged yet?
if "transcript" not in labels_df.columns:
    print("No 'transcript' column in labels_df; nothing to peek.")
else:
    # 1) Coverage summary (how many non-empty transcripts do we actually have?)
    tx_series = labels_df["transcript"].astype(str)
    nonempty_mask = tx_series.str.len() > 0
    tx = tx_series[nonempty_mask].tolist()
    print(f"Non-empty transcripts: {nonempty_mask.sum()} / {len(labels_df)}")

    if not tx:
        print("All transcripts are empty - skipping IDF peek.")
    else:
        # 2) Refit a tiny TF-IDF with the SAME settings used in 3.2
        #    (We only need the fitted vocabulary + IDF weights for transparency.)
        vec = TfidfVectorizer(
            max_features=TFIDF_MAX_FEATS,
            ngram_range=TFIDF_NGRAMS,
            min_df=TFIDF_MIN_DF,
            stop_words="english",
            strip_accents="unicode",
            dtype=np.float32,
        )
        X = vec.fit_transform(tx)                         # fit only on non-empty docs
        vocab = vec.get_feature_names_out().tolist()      # learned terms (size <= max_features)
        idf_vals = vec.idf_.astype(float)                 # IDF weights; higher = rarer

        idf_table = (
            pd.DataFrame({"term": vocab, "idf": idf_vals})
            .sort_values("idf", ascending=False)          # rarest first
            .reset_index(drop=True)
        )

        # 3) Show a small slice for reviewers
        print("\nTop 15 highest-IDF (rarest) terms:")
        print(idf_table.head(15).to_string(index=False))

        print("\nBottom 15 lowest-IDF (most common) terms:")
        print(idf_table.tail(15).to_string(index=False))

        # 4) "Spidery" quick check (very short tokens or tokens with odd characters)
        spidery_mask = idf_table["term"].str.match(r"(^.{,2}$|.*[^a-z0-9_'\-].*)", case=False)
        spidery = idf_table[spidery_mask]
        if not spidery.empty:
            print(f"\nHeads-up: {len(spidery)} suspicious terms (very short or odd chars).")
            print(spidery.head(10).to_string(index=False))

        # 5) Save full IDF table for provenance
        TEXT_IDF_CSV = PROCESSED_DIR / "text_idf_terms.csv"
        idf_table.to_csv(TEXT_IDF_CSV, index=False)
        print("\nSaved IDF table ->", TEXT_IDF_CSV, "| shape =", idf_table.shape)



---
**How to read this (TF-IDF IDF transparency)**
- **High IDF** = rarer terms  often more distinctive for a document; review to ensure no leaky or sensitive tokens.
- **Low IDF** = very common terms  candidates for stoplist if they're uninformative in this corpus.
- **Spidery terms** (very short / odd characters)  likely typos, artifacts, or noise; consider cleaning or adding to a custom stoplist.
- We saved `data/processed/text_idf_terms.csv` so others can audit terms across runs.
---


In [ ]:
# =============================================================================
# 3.2c Custom-stoplist TF-IDF (parallel artifacts for comparison)
# -----------------------------------------------------------------------------
# What:
#    Adds conversational fillers & obvious noise to a custom stoplist
#    Normalizes stopwords with the SAME analyzer TF-IDF uses (no warnings)
#    Rebuilds TF-IDF (same settings as 3.2) and saves parallel artifacts
# Why:
#    Remove uninformative speech tokens; keep features focused on content
#    Side-by-side artifacts let us compare base vs custom stoplists
# =============================================================================

import json, numpy as np, pandas as pd
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

# Guard
if "transcript" not in labels_df.columns:
    print("No 'transcript' column available; skipping custom TF-IDF.")
else:
    tx_series = labels_df["transcript"].astype(str)
    nonempty_mask = tx_series.str.len() > 0
    nonempty = tx_series[nonempty_mask].tolist()
    if not nonempty:
        print("All transcripts empty; skipping custom TF-IDF.")
    else:
        # --- 1) Conversational fillers / noises to filter (extend as needed)
        CUSTOM_STOPS = {
            "um","umm","uh","uhh","uhhh","ah","oh","hmm","hmmm","mmm",
            "yeah","yep","nope","ok","okay",
            "like","just","really","kinda","sorta","ya","yall","y'all",
            "you","youre","you're","youknow","ya know","you know",
            "i","im","i'm","id","i'd","ive","i've","me","my","mine",
            "uh-huh","huh","mm-hmm",
            "laughter"  # appears very frequently in this corpus
        }

        # --- 2) Build a temporary vectorizer to get the SAME analyzer as our TF-IDF
        _tmp_vec = TfidfVectorizer(
            ngram_range=TFIDF_NGRAMS,
            min_df=TFIDF_MIN_DF,
            stop_words=None,            # no stops yet
            strip_accents="unicode",
            dtype=np.float32,
        )
        analyzer = _tmp_vec.build_analyzer()

        # Merge sklearn's English stops + our conversational stops, then NORMALIZE with the analyzer
        custom_only   = {s.lower() for s in CUSTOM_STOPS}
        raw_stopset   = set(ENGLISH_STOP_WORDS) | custom_only

        norm_stopset = set()
        for s in raw_stopset:
            # e.g., "I've" -> ["ve"], "mm-hmm" -> ["mm","hmm"]
            for tok in analyzer(s):
                norm_stopset.add(tok)

        STOPLIST = sorted(norm_stopset)  # list-like, deterministic
        added_beyond_english = len(custom_only - set(ENGLISH_STOP_WORDS))
        print(f"Custom stoplist (normalized) size: {len(STOPLIST)} (added {added_beyond_english} beyond sklearn English)")

        # --- 3) Vectorize with the SAME settings as 3.2, but using our STOPLIST
        vec_custom = TfidfVectorizer(
            max_features=TFIDF_MAX_FEATS,
            ngram_range=TFIDF_NGRAMS,
            min_df=TFIDF_MIN_DF,
            stop_words=STOPLIST,
            strip_accents="unicode",
            dtype=np.float32,
        )
        Xc = vec_custom.fit_transform(nonempty)
        vocab_c = vec_custom.get_feature_names_out().tolist()

        # --- 4) Dense float32 for Parquet
        arr = Xc.toarray().astype("float32")
        tfidf_cols_c = [f"tfidf_{t}" for t in vocab_c]
        tfidf_custom = pd.DataFrame(arr, columns=tfidf_cols_c)

        # Align IDs to the same non-empty mask used above
        ids_nonempty = labels_df.loc[nonempty_mask, JOIN_KEY].to_numpy()
        tfidf_custom.insert(0, JOIN_KEY, ids_nonempty)

        # --- 5) Save parallel artifacts
        TEXT_TFIDF_CUSTOM = PROCESSED_DIR / "text_tfidf_custom.parquet"
        TEXT_VOCAB_CUSTOM = PROCESSED_DIR / "text_tfidf_vocab_custom.json"
        TEXT_IDF_CUSTOM   = PROCESSED_DIR / "text_idf_terms_custom.csv"

        tfidf_custom.to_parquet(TEXT_TFIDF_CUSTOM, index=False, engine="pyarrow", compression="snappy")
        Path(TEXT_VOCAB_CUSTOM).write_text(json.dumps(
            {
                "vocab": vocab_c,
                "ngram_range": TFIDF_NGRAMS,
                "min_df": TFIDF_MIN_DF,
                "custom_stops": sorted(list(CUSTOM_STOPS)),  # human-readable list we started with
            }
        ))

        idf_c = vec_custom.idf_.astype(float)
        idf_table_c = pd.DataFrame({"term": vocab_c, "idf": idf_c}).sort_values("idf", ascending=False)
        idf_table_c.to_csv(TEXT_IDF_CUSTOM, index=False)

        print(f"Saved TF-IDF (custom) -> {TEXT_TFIDF_CUSTOM} | shape={tfidf_custom.shape}")
        print(f"Saved TF-IDF vocab (custom) -> {TEXT_VOCAB_CUSTOM} | {len(vocab_c)} terms")
        print(f"Saved IDF table (custom) -> {TEXT_IDF_CUSTOM} | shape={idf_table_c.shape}")



---
### 3.2d Visual: Top-IDF terms (base vs custom)
Quick look at the rarest (highest-IDF) terms under the baseline and custom-stoplist runs.

---


In [ ]:
# 3.2d Visual: top-IDF bars (base vs custom)
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

base_idf = PROCESSED_DIR / "text_idf_terms.csv"
cust_idf = PROCESSED_DIR / "text_idf_terms_custom.csv"

if base_idf.exists() and cust_idf.exists():
    base_df = pd.read_csv(base_idf).sort_values("idf", ascending=False).head(15)
    cust_df = pd.read_csv(cust_idf).sort_values("idf", ascending=False).head(15)

    # Base (top 15 rarest)
    ax = base_df.sort_values("idf").plot(kind="barh", x="term", y="idf", legend=False)
    ax.set_title("Top 15 highest-IDF terms - BASE")
    ax.set_xlabel("IDF"); ax.set_ylabel("term")
    plt.tight_layout(); plt.show()

    # Custom (top 15 rarest)
    plt.figure()
    ax = cust_df.sort_values("idf").plot(kind="barh", x="term", y="idf", legend=False)
    ax.set_title("Top 15 highest-IDF terms - CUSTOM stoplist")
    ax.set_xlabel("IDF"); ax.set_ylabel("term")
    plt.tight_layout(); plt.show()

    base_terms   = set(base_df["term"])
    custom_terms = set(cust_df["term"])
    print("Rarest terms unique to BASE:  ", sorted(base_terms - custom_terms)[:10])
    print("Rarest terms unique to CUSTOM:", sorted(custom_terms - base_terms)[:10])
else:
    print("Missing one of:", base_idf, cust_idf)


---
### 3.2e Visual: Transcript length distribution (QC)
How many tokens per transcript? Outliers or very short transcripts stand out here.


In [ ]:
# 3.2e Visual: histogram of transcript token lengths
import pandas as pd
import matplotlib.pyplot as plt

meta_path = PROCESSED_DIR / "text_meta.parquet"
if meta_path.exists():
    meta = pd.read_parquet(meta_path)
    ax = meta["text_len_tokens"].dropna().plot(kind="hist", bins=20)
    ax.set_title("Transcript token length distribution")
    ax.set_xlabel("tokens per transcript")
    plt.tight_layout(); plt.show()

    print("Summary stats:")
    print(meta["text_len_tokens"].describe())
else:
    print("Missing:", meta_path)


---
### 3.2f Visual: PHQ-8 vs transcript length
Sanity correlation (not causal): do longer transcripts co-vary with PHQ-8?

---

In [ ]:
# 3.2f Visual: PHQ-8 vs transcript length scatter with Pearson r
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

tab_path  = PROCESSED_DIR / "tabular_phq8.parquet"
meta_path = PROCESSED_DIR / "text_meta.parquet"

if tab_path.exists() and meta_path.exists():
    phq  = pd.read_parquet(tab_path)[[JOIN_KEY, "phq8_sum"]]
    meta = pd.read_parquet(meta_path)[[JOIN_KEY, "text_len_tokens"]]
    merged = phq.merge(meta, on=JOIN_KEY, how="inner").dropna()

    if len(merged) >= 10:
        r, p = pearsonr(merged["phq8_sum"].astype(float), merged["text_len_tokens"].astype(float))
        ax = merged.plot(kind="scatter", x="text_len_tokens", y="phq8_sum", alpha=0.6)
        ax.set_title(f"PHQ-8 vs Transcript Length  (r={r:.3f}, p={p:.3g}, n={len(merged)})")
        ax.set_xlabel("tokens per transcript"); ax.set_ylabel("PHQ-8 sum")
        plt.tight_layout(); plt.show()

        print(merged[[JOIN_KEY, "phq8_sum", "text_len_tokens"]].head(5))
    else:
        print("Not enough rows for correlation (n<10). Current n =", len(merged))
else:
    print("Missing one of:", tab_path, meta_path)


---
### 3.2g Visual: Vocab overlap (base vs custom)
How much of the vocabulary is shared vs unique across the two runs?


In [ ]:
# 3.2g Visual: base vs custom vocab overlap
import json, matplotlib.pyplot as plt

base_meta_path   = PROCESSED_DIR / "text_tfidf_vocab.json"
custom_meta_path = PROCESSED_DIR / "text_tfidf_vocab_custom.json"

if base_meta_path.exists() and custom_meta_path.exists():
    base_meta   = json.loads(base_meta_path.read_text())
    custom_meta = json.loads(custom_meta_path.read_text())
    base_terms   = set(base_meta.get("vocab", []))
    custom_terms = set(custom_meta.get("vocab", []))

    only_base   = len(base_terms - custom_terms)
    only_custom = len(custom_terms - base_terms)
    both        = len(base_terms & custom_terms)

    plt.figure()
    plt.bar(["base only", "both", "custom only"], [only_base, both, only_custom])
    plt.title("Vocab overlap: base vs custom stoplist")
    plt.ylabel("terms")
    plt.tight_layout(); plt.show()

    print(f"base |V|={len(base_terms)}, custom |V|={len(custom_terms)}, both={both}")
else:
    print("Missing one of:", base_meta_path, custom_meta_path)


---
### 3.2 Narrative - What was done & why

**Goal.** Build transparent text features from interview transcripts that we can audit and reproduce.

**What we did.**
- **3.2a Import & join.** Discovered per-participant `*_TRANSCRIPT.csv` files, concatenated a participant's utterances  one `transcript` string per `participant_id`, and merged into `labels_df`.
- **3.2 TF-IDF baseline + QC.** 
  - Cleaned text lightly (whitespace/accents).
  - Built **TF-IDF** with `max_features=2048`, `ngram_range=(1,2)`, `min_df=2`, English stopwords; saved:
    - `text_tfidf.parquet` (dense float32 features) and `text_tfidf_vocab.json` (vocabulary + settings).
  - Wrote **QC features** per participant (`text_len_chars`, `text_len_tokens`, `text_num_sentences`) to `text_meta.parquet`.
- **3.2c Spider check (transparency).** Refit TF-IDF only to read the **IDF table** (rare  common terms), flagged "spidery" tokens (very short/odd), and saved `text_idf_terms.csv` for provenance.
- **3.2d Custom stoplist.** Added a small set of conversational fillers and obvious noise (um/uh/like/just/...); **normalized** the stoplist using the same analyzer TF-IDF uses (no inconsistency warnings). Rebuilt a parallel TF-IDF and saved:
  - `text_tfidf_custom.parquet`, `text_tfidf_vocab_custom.json`, `text_idf_terms_custom.csv`.
- **Section 4 merge.** Assembled a **multimodal** table with PHQ-8 (`tab_*`), text QC (`txt_*`), base TF-IDF (`tfidf_*`), and custom TF-IDF (`tfidfC_*`).

**Why this design.**
- **Transparency:** IDF tables + QC stats make structure visible to reviewers.
- **Reproducibility:** sidecar vocab JSONs and saved IDF CSVs document exactly what was used.
- **Graceful failure:** SKIP-safe cells keep nbconvert/CI green even if a modality isn't ready.


---
### 3.2 Results & Key Takeaways

**Coverage & QC.**
- **Non-empty transcripts:** 110 / 110 (in the spider-check set).
- **Transcript length (tokens):** mean  **1410**, std  **787**, min **78**, max **4116**. Most interviews fall in a mid-length band; very short transcripts are potential QC outliers.
- **PHQ-8 vs length:** Pearson **r = 0.103**, **p = 0.283**, **n = 110**  weak, non-significant association (descriptive only, not causal).

**Vocabulary & IDF.**
- **Baseline TF-IDF:** 2048 terms (cap), unigrams+bigrams with English stopwords.
- **Custom stoplist TF-IDF:** 2048 terms (cap) after removing fillers (um/uh/like/just/...).
- **Overlap:** **both  1355** terms; **base-only  693**, **custom-only  693**  the custom list trims conversational filler without collapsing the feature space.
- **Rarest (high-IDF) terms:** surface topical words (examples from this run included items like "coughs", "mountain", "marketing", "prison", etc.); good places to check for typos/sensitive tokens.
- **Most common (low-IDF) terms:** generic/filler ("things", "laughter", "think", "people", "know", "really", "like", "um/uh/just"); strong candidates for stoplisting.

**What changed with the custom stoplist.**
- Frequent filler tokens were removed/discounted; bar charts show rarer **topical** terms move up the IDF ranks.
- Vocab **stability** remained high ( two-thirds shared), indicating a modest, targeted refocus rather than a wholesale shift.

**Artifacts (reproducible).**
- Baseline: `text_meta.parquet`, `text_tfidf.parquet`, `text_tfidf_vocab.json`, `text_idf_terms.csv`
- Custom: `text_tfidf_custom.parquet`, `text_tfidf_vocab_custom.json`, `text_idf_terms_custom.csv`
- Merged: `multimodal_features.parquet` with `tab_*`, `txt_*`, `tfidf_*`, `tfidfC_*`

**Limitations / notes.**
- TF-IDF is bag-of-words; no syntax/semantics. We'll consider **sentence embeddings** later if resources allow.
- Stoplists reduce noise but are corpus-specific; we'll keep an eye on "spidery" terms and adjust iteratively.

**Next steps.**
- (A) Optional: refine custom stops (e.g., domain-specific fillers), re-run 3.2d and compare IDF tables.
- (B) Proceed to **3.3 Audio** (prosody aggregates) and **3.4 Video** (AU/gaze aggregates).
- (C) Train quick baselines on text-only (TF-IDF logistic) vs tabular-only vs multimodal to quantify lift.

---


### 3.2 Executive Summary (Text Features)

We ingested and joined per-participant transcripts, then built a transparent TF-IDF baseline (unigram+bigram, 2K cap) with QC stats; coverage is 110/110 non-empty transcripts. A custom stoplist (normalized to the analyzer) removed conversational fillers (um/uh/like/just/...), yielding a parallel TF-IDF that preserved vocabulary stability (1,355 shared terms; balanced base-only vs custom-only) while surfacing more topical rare terms. Descriptively, transcript length showed a weak, non-significant association with PHQ-8 (r  0.10, p  0.28), suggesting verbosity isn't tightly linked to severity in this set. All artifacts (vocab JSONs, IDF tables, features) are saved for reproducibility, and both BASE (`tfidf_*`) and CUSTOM (`tfidfC_*`) features are included in the merged multimodal table for downstream modeling.

---


In [ ]:
# --- DIAGNOSTIC: find a root, list counts, show sample columns (audio + video) ---
from pathlib import Path
import pandas as pd

# 0) Confirm ROOT_DIR (used everywhere in NB03)
try:
    print("ROOT_DIR =", ROOT_DIR)
except NameError:
    ROOT_DIR = Path.cwd().resolve().parent
    print("ROOT_DIR not set; using parent of notebooks/:", ROOT_DIR)

# 1) Pick a search root (prefer folders you actually showed in screenshots)
CAND_ROOTS = [
    ROOT_DIR / "data" / "daic_woz",
    ROOT_DIR / "data" / "raw" / "daic_woz",
    ROOT_DIR / "data" / "raw",
    ROOT_DIR / "data",
]

ROOT = next((r for r in CAND_ROOTS if r.exists()), ROOT_DIR / "data")
print("Search ROOT:", ROOT)

# 2) Collect matches (don't print paths yet; just counts)
aud_cov   = list(ROOT.rglob("*COVAREP.csv"))
aud_form  = list(ROOT.rglob("*FORMANT.csv"))
vid_clnf  = list(ROOT.rglob("*CLNF_features.*"))
vid_openf = list(ROOT.rglob("*OpenFace*.csv"))
vid_aus   = list(ROOT.rglob("*_AUs.csv"))

print("\nCounts:")
print(" audio COVAREP:", len(aud_cov))
print(" audio FORMANT:", len(aud_form))
print(" video CLNF_features.*:", len(vid_clnf))
print(" video OpenFace*.csv:", len(vid_openf))
print(" video *_AUs.csv:", len(vid_aus))

# 3) Show 2 audio + 2 video examples (paths + first 12 columns)
def _peek_csv(path: Path):
    for kwargs in ({"engine":"python","sep":None},{"sep":"\t"},{"sep":","},{"engine":"python","sep":r"\s+"}):
        for enc in ("utf-8","latin-1"):
            try:
                return pd.read_csv(path, encoding=enc, **kwargs, nrows=3)
            except Exception:
                continue
    return None

print("\nSample audio files:")
for p in (aud_cov[:1] + aud_form[:1]):
    print(" ", p)
    df = _peek_csv(p)
    print("   cols:", list(df.columns)[:12] if df is not None else "unreadable")

print("\nSample video files:")
for p in (vid_clnf[:1] + vid_openf[:1] + vid_aus[:1])[:2]:
    print(" ", p)
    df = _peek_csv(p)
    print("   cols:", list(df.columns)[:12] if df is not None else "unreadable")

print("\n(If these are all zero, verify the folder path in CAND_ROOTS matches where your files live.)")



---
### 3.3 Audio (prosody)
- Fundamental frequency (f0), jitter/shimmer, loudness/energy, spectral features. 
- Extract with OpenSMILE or COVAREP, then aggregate per session (mean, std, percentiles).

---


In [ ]:
# =============================================================================
# 3.3 Audio (prosody) - per-participant aggregates (robust, SKIP-safe, verbose)
# =============================================================================
import sys, time, re, pandas as pd, numpy as np
from pathlib import Path

AUDIO_OUT = PROCESSED_DIR / "audio_features.parquet"

# --- Verbosity controls ---
VERBOSE_EVERY = 25   # print a line every N files
LIMIT = None         # set to e.g. 30 for a quick dry-run, then None for full run

# Candidate roots (pick first that exists)
CAND_ROOTS = [
    ROOT_DIR / "data" / "daic_woz",
    ROOT_DIR / "data" / "raw" / "daic_woz",
    ROOT_DIR / "data" / "raw",
    ROOT_DIR / "data",
]
RAW_DIR = next((r for r in CAND_ROOTS if r.exists()), ROOT_DIR / "data")
print("Audio search ROOT:", RAW_DIR)

def _pid_from_stem(stem: str) -> str:
    m = re.match(r"^(\d+)", stem)
    return m.group(1) if m else stem

def _read_csv_any(path: Path) -> pd.DataFrame | None:
    for kwargs in ({"engine":"python","sep":None},{"sep":"\t"},{"sep":","},{"engine":"python","sep":r"\s+"}):
        for enc in ("utf-8","latin-1"):
            try:
                return pd.read_csv(path, encoding=enc, **kwargs, low_memory=False)
            except Exception:
                continue
    return None

audio_files = list(RAW_DIR.rglob("*COVAREP.csv")) + list(RAW_DIR.rglob("*FORMANT.csv"))
print(f"Found {len(audio_files)} audio feature file(s)")
if LIMIT:
    audio_files = audio_files[:LIMIT]
    print(f"DRY-RUN: limiting to first {len(audio_files)} files")

rows, t0 = [], time.time()
for i, p in enumerate(audio_files, 1):
    df = _read_csv_any(p)
    if df is None or df.empty:
        print("SKIP: unreadable/empty audio file ->", p.name); continue

    # Coerce numbers robustly
    num = df.apply(pd.to_numeric, errors="coerce").select_dtypes(include=[np.number])
    if num.empty:
        print("SKIP: no numeric columns in", p.name); continue

    part = _pid_from_stem(p.stem)
    src  = "COVAREP" if "COVAREP" in p.name.upper() else "FORMANT"
    agg = {
        **num.mean(numeric_only=True).add_prefix("mean_").to_dict(),
        **num.std(numeric_only=True).add_prefix("std_").to_dict(),
        **num.median(numeric_only=True).add_prefix("med_").to_dict(),
        **num.quantile(0.10, numeric_only=True).add_prefix("p10_").to_dict(),
        **num.quantile(0.90, numeric_only=True).add_prefix("p90_").to_dict(),
        "n_frames": len(num),
        "source": src,
    }
    rows.append({"participant_id": part, **agg})

    if (i % VERBOSE_EVERY) == 0 or i == 1:
        dt = time.time() - t0
        print(f"[audio] processed {i}/{len(audio_files)} ... ({dt:.1f}s)")
        sys.stdout.flush()

audio_df = pd.DataFrame(rows)
if audio_df.empty:
    print("SKIP: no audio features aggregated.")
else:
    audio_agg = audio_df.groupby("participant_id").mean(numeric_only=True).reset_index()
    audio_agg.to_parquet(AUDIO_OUT, index=False)
    print("Saved audio features ->", AUDIO_OUT, "| shape=", audio_agg.shape, "| total time:", f"{time.time()-t0:.1f}s")




---
## 🕷️ Spider Check - Audio Features Peek

Listening between the lines: prosody features like pitch, shimmer, and formants.  
A quick peace-of-mind peek at participant audio stats to be sure the signals look right.  

***Because voices carry more than words - they carry what's unseen.***

---


In [ ]:
from pathlib import Path
import pandas as pd, numpy as np

AUDIO_OUT = PROCESSED_DIR / "audio_features.parquet"
audio_agg = pd.read_parquet(AUDIO_OUT)

print("Audio agg shape:", audio_agg.shape)
print("Nulls (top 10):")
print(audio_agg.isna().sum().sort_values(ascending=False).head(10))

# ensure participant_id is str (helps merges)
audio_agg["participant_id"] = audio_agg["participant_id"].astype(str)

# quick peek
display(audio_agg.head(3).iloc[:, :12])  # first dozen cols


---
## 🕷️ Spider Check - Multimodal Merge Peek

Weaving the web: audio, text, and tabular features come together here.  
A quick integrity check that strands align by participant_id before modeling.  

***Because the strength of the web depends on what's unseen between the strands.***

---


In [ ]:
PEEK = True  # set False for speed; True for a quick spider check


In [ ]:
# =============================================================================
# Data Inventory - fast summary with optional peek()
# =============================================================================
import os
from pathlib import Path
import pandas as pd

proc_dir = ROOT_DIR / "data" / "processed"

FILES = [
    "audio_features.parquet",
    "text_meta.parquet",
    "text_tfidf.parquet",
    "text_tfidf_custom.parquet",
    "text_idf_terms.csv",
    "text_idf_terms_custom.csv",
    "transcripts_unified.csv",
    "tabular_phq8.parquet",
    "multimodal_features.parquet",
]

# ---- knobs ----
PEEK = False          # flip to True when you want head(2)
MAX_PEEK_COLS = 8     # cap preview width for mega-wide frames
ENGINE = "pyarrow"    # or "fastparquet" if you prefer

def _read(path: Path) -> pd.DataFrame:
    if path.suffix == ".parquet":
        return pd.read_parquet(path, engine=ENGINE)
    return pd.read_csv(path)

def describe(path: Path, peek: bool = False) -> dict:
    if not path.exists():
        return {"exists": False}
    try:
        df = _read(path)
        info = {
            "exists": True,
            "shape": df.shape,
            "columns": df.columns[:5].tolist(),
            "non_nulls_top": df.notna().sum().sort_values(ascending=False).head(5).to_dict(),
            "file_size_mb": round(os.path.getsize(path) / (1024**2), 2),
        }
        if peek:
            cols = df.columns[:MAX_PEEK_COLS]
            info["peek"] = df.loc[:, cols].head(2).to_dict(orient="records")
        return info
    except Exception as e:
        return {"exists": True, "error": str(e)}

inv = {name: describe(proc_dir / name, peek=PEEK) for name in FILES}
pd.set_option("display.max_colwidth", 140)
pd.DataFrame(inv).T



---
### 3.4 Video (facial action units)
- Use OpenFace to extract per frame AUs and gaze; aggregate per session. 
- Typical aggregates: mean, std, max, fraction above threshold.


In [ ]:
# =============================================================================
# 3.4 Video (AUs/gaze) - per-participant aggregates (SKIP-safe)
# -----------------------------------------------------------------------------
# Looks for files like ".../123_OpenFace.csv" or ".../123_AUs.csv"
# Aggregates AU* and gaze_* numeric columns per participant (mean, std, median, p10/p90, #frames)
# Saves to data/processed/video_features.parquet
# =============================================================================
import re, pandas as pd, numpy as np
from pathlib import Path

VIDEO_OUT = PROCESSED_DIR / "video_features.parquet"
RAW_DIR   = ROOT_DIR / "data"

def _pid_from_stem(stem: str) -> str:
    m = re.match(r"^(\d+)", stem)
    return m.group(1) if m else stem

video_files = list(RAW_DIR.rglob("*OpenFace*.csv")) + list(RAW_DIR.rglob("*_AUs.csv"))
print(f"Found {len(video_files)} video feature file(s)")

rows = []
for p in video_files:
    try:
        df = pd.read_csv(p)
    except Exception:
        try:
            df = pd.read_csv(p, encoding="latin-1")
        except Exception:
            print("SKIP: unreadable video file ->", p.name); continue

    # keep columns typically exported by OpenFace: AU*, gaze_*, head pose etc.
    num = df.select_dtypes(include=[np.number])
    # prefer AU/gaze subset if present
    keep_cols = [c for c in num.columns if c.startswith(("AU", "gaze", "pose", "confidence"))]
    num = num[keep_cols] if keep_cols else num
    if num.empty:
        print("SKIP: no AU/gaze numeric columns in", p.name); continue

    part = _pid_from_stem(p.stem)
    agg = {
        **num.mean(numeric_only=True).add_prefix("mean_").to_dict(),
        **num.std(numeric_only=True).add_prefix("std_").to_dict(),
        **num.median(numeric_only=True).add_prefix("med_").to_dict(),
        **num.quantile(0.10, numeric_only=True).add_prefix("p10_").to_dict(),
        **num.quantile(0.90, numeric_only=True).add_prefix("p90_").to_dict(),
        "n_frames": len(num),
    }
    rows.append({"participant_id": part, **agg})

video_df = pd.DataFrame(rows)
if video_df.empty:
    print("SKIP: no video features aggregated.")
else:
    video_agg = video_df.groupby("participant_id").mean(numeric_only=True).reset_index()
    video_agg.to_parquet(VIDEO_OUT, index=False)
    print("Saved video features ->", VIDEO_OUT, "| shape=", video_agg.shape)




## 4) Multimodal dataset assembly

Merge per-modality feature tables on `['subject_id','session_id']`, align with labels, handle missing data, and validate splits.


In [ ]:
# =============================================================================
# 4) Multimodal dataset assembly (tolerant to missing modalities)
# Goal:
#   - Merge per-modality feature tables by JOIN_KEY.
#   - Handle absent files gracefully (print+SKIP).
#   - Produce one row per participant with clear column prefixes.
# =============================================================================

import pandas as pd
from pathlib import Path

MM_OUT = PROCESSED_DIR / "multimodal_features.parquet"

def _safe_read_parquet(path: Path, note: str) -> pd.DataFrame:
    """Read a parquet table if present; otherwise return an empty DF with JOIN_KEY."""
    if path.exists():
        try:
            df = pd.read_parquet(path)
            print(f"Loaded {note} -> {path} | shape={df.shape}")
            return df
        except Exception as e:
            print(f"SKIP read {note}: {type(e).__name__} - {e}")
    else:
        print(f"SKIP: {note} not found at {path}")
    return pd.DataFrame({JOIN_KEY: pd.Series(dtype="object")})

def _prefix(df: pd.DataFrame, prefix: str, skip_cols=(JOIN_KEY,)):
    """Prefix all non-ID columns to keep namespaces tidy."""
    if df.empty:
        return df
    cols = {c: f"{prefix}{c}" for c in df.columns if c not in skip_cols}
    return df.rename(columns=cols)

# ---- Core labels (ID + target only) -----------------------------------------
core = labels_df[[JOIN_KEY, TARGET]].drop_duplicates(subset=[JOIN_KEY]).copy()

# ---- Tabular PHQ-8 ----------------------------------------------------------
tab_p = _safe_read_parquet(PROCESSED_DIR / "tabular_phq8.parquet", "PHQ-8 tabular")
# Drop duplicate target from tabular if present
tab_p = tab_p.drop(columns=[c for c in [TARGET] if c in tab_p.columns], errors="ignore")

# ---- Text meta + base TF-IDF ------------------------------------------------
tx_meta  = _safe_read_parquet(PROCESSED_DIR / "text_meta.parquet",  "text meta")
tx_tfidf = _safe_read_parquet(PROCESSED_DIR / "text_tfidf.parquet", "text TF-IDF")

# ---- (NEW) Custom-stoplist TF-IDF -------------------------------------------
tx_tfidf_custom = _safe_read_parquet(PROCESSED_DIR / "text_tfidf_custom.parquet", "text TF-IDF (custom)")

# ---- Prefix columns to avoid collisions -------------------------------------
tab_p           = _prefix(tab_p,           "tab_")
tx_meta         = _prefix(tx_meta,         "txt_")
tx_tfidf        = _prefix(tx_tfidf,        "tfidf_")
tx_tfidf_custom = _prefix(tx_tfidf_custom, "tfidfC_")

# ---- Merge everything on JOIN_KEY -------------------------------------------
mm = core.merge(tab_p,           on=JOIN_KEY, how="left")
mm = mm.merge(tx_meta,           on=JOIN_KEY, how="left")
mm = mm.merge(tx_tfidf,          on=JOIN_KEY, how="left")
mm = mm.merge(tx_tfidf_custom,   on=JOIN_KEY, how="left")  # add custom TF-IDF

# ---- Final sanity + save -----------------------------------------------------
print("Multimodal shape:", mm.shape)
mm.to_parquet(MM_OUT, index=False)
print("Saved multimodal ->", MM_OUT)

# Small preview: ID, target, a few PHQ-8 features, and text QC columns
peek_cols = [JOIN_KEY, TARGET]
peek_cols += [c for c in mm.columns if c.startswith("tab_phq8_")][:5]
peek_cols += [c for c in mm.columns if c.startswith("txt_text_len_")][:2]
print("Preview:")
print(mm[[c for c in peek_cols if c in mm.columns]].head(5))




## 5) Artifacts (saved processed data)
Save per modality tables and the merged multimodal dataset for downstream modeling.


In [ ]:

ART_TEXT = PROC_DIR / 'text_embeddings.parquet'
ART_AUDIO = PROC_DIR / 'audio_features.parquet'
ART_VIDEO = PROC_DIR / 'video_features.parquet'
ART_TAB = PROC_DIR / 'phq8_engineered.parquet'
ART_MERGE = PROC_DIR / 'multimodal_merged.parquet'

# Save only if non-empty (avoid writing empty placeholder frames)
if not df_tab.empty: df_tab.to_parquet(ART_TAB, index=False)
if not df_text_model.empty: df_text_model.to_parquet(ART_TEXT, index=False)
if not df_audio.empty: df_audio.to_parquet(ART_AUDIO, index=False)
if not df_video.empty: df_video.to_parquet(ART_VIDEO, index=False)
if not df_features.empty: df_features.to_parquet(ART_MERGE, index=False)

print('Saved (if available):')
for p in [ART_TAB, ART_TEXT, ART_AUDIO, ART_VIDEO, ART_MERGE]:
 print('-', p, p.exists())



## 6) Limitations & reproducibility

**Limitations**
- Placeholder tables for text/audio/video until extraction pipelines are connected. 
- Class imbalance persists; monitor PR curves and calibration in later notebooks. 
- Alignment between modalities may vary by session; verify IDs and time boundaries upstream.

**Reproducibility**
- Python 3.11 (`venv`) 
- Core libs: `pandas`, `numpy`, `scikit-learn`, `matplotlib`, `z3-solver` 
- Optional libs: `sentence-transformers`, `librosa`/`opensmile`, `openface` (CLI) 



## 7) Closing summary & next steps

- Engineered tabular features (PHQ 8 with standardized interactions). 
- Added SMT guardrails to catch data/timing/split issues early. 
- Assembled a merged, subject-disjoint multimodal dataset ready for modeling.

**Next:** Notebook 04 - Multimodal Modeling & Fusion (LR/RF/baseline NN; late fusion vs. early fusion; calibration & interpretability).
